In [2]:
import numpy as np
import pandas as pd
import sys
import re
import time
import datetime
from colorama import init, Fore, Back, Style
from time import time, gmtime, strftime
from datetime import datetime
from hyperdash import monitor_cell  
from tqdm import tqdm

from sklearn.metrics import cohen_kappa_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize as tokenizer


from IPython.core.interactiveshell import InteractiveShell # no need of print for several objects!!!
InteractiveShell.ast_node_interactivity = "all"

#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

## Iterate over XML files for random draw of samples

In [11]:
full_tsv_list = ["00-meta-history1.xml-p3p3661.tsv","01-meta-history1.xml-p3662p10075.tsv","02-meta-history1.xml-p10076p18526.tsv","03-meta-history1.xml-p18527p30361.tsv","04-meta-history1.xml-p30362p43904.tsv","05-meta-history1.xml-p43905p56252.tsv","06-meta-history1.xml-p56253p71552.tsv","07-meta-history1.xml-p71553p89754.tsv","08-meta-history1.xml-p89755p107570.tsv","09-meta-history1.xml-p107571p128187.tsv",
"10-meta-history1.xml-p128188p150477.tsv","11-meta-history1.xml-p150478p174227.tsv","12-meta-history1.xml-p174228p196344.tsv","13-meta-history1.xml-p196345p222500.tsv","14-meta-history1.xml-p222501p250312.tsv","15-meta-history1.xml-p250313p280868.tsv","16-meta-history1.xml-p280869p307896.tsv","17-meta-history1.xml-p307897p329059.tsv","18-meta-history1.xml-p329060p356265.tsv","19-meta-history1.xml-p356266p389955.tsv",
"20-meta-history1.xml-p389956p412301.tsv","21-meta-history2.xml-p412304p449997.tsv","22-meta-history2.xml-p449998p485915.tsv","23-meta-history2.xml-p485916p527163.tsv","24-meta-history2.xml-p527164p564312.tsv","25-meta-history2.xml-p564313p612957.tsv","26-meta-history2.xml-p612958p662695.tsv","27-meta-history2.xml-p662696p715915.tsv","28-meta-history2.xml-p715916p770350.tsv","29-meta-history2.xml-p770351p824123.tsv",
"30-meta-history2.xml-p824124p876600.tsv","31-meta-history2.xml-p876601p926793.tsv","32-meta-history2.xml-p926794p985318.tsv","33-meta-history2.xml-p985319p1045696.tsv","34-meta-history2.xml-p1045697p1113860.tsv","35-meta-history2.xml-p1113861p1186656.tsv","36-meta-history2.xml-p1186657p1252585.tsv","37-meta-history2.xml-p1252586p1312869.tsv","38-meta-history2.xml-p1312870p1382438.tsv","39-meta-history2.xml-p1382439p1454699.tsv",
"40-meta-history2.xml-p1454700p1557686.tsv","41-meta-history2.xml-p1557687p1647888.tsv","42-meta-history3.xml-p1647895p1740137.tsv","43-meta-history3.xml-p1740138p1843398.tsv","44-meta-history3.xml-p1843399p1939691.tsv","45-meta-history3.xml-p1939692p2031436.tsv","46-meta-history3.xml-p2031437p2286439.tsv","47-meta-history3.xml-p2286440p2501692.tsv","48-meta-history3.xml-p2501693p2730817.tsv","49-meta-history3.xml-p2730818p2832611.tsv",
"50-meta-history3.xml-p2832612p2941309.tsv","51-meta-history3.xml-p2941310p3055771.tsv","52-meta-history3.xml-p3055772p3166492.tsv","53-meta-history3.xml-p3166493p3305127.tsv","54-meta-history3.xml-p3305128p3438516.tsv","55-meta-history3.xml-p3438517p3561507.tsv","56-meta-history3.xml-p3561508p3682191.tsv","57-meta-history3.xml-p3682192p3796516.tsv","58-meta-history3.xml-p3796517p3922524.tsv","59-meta-history3.xml-p3922525p4040465.tsv",
"60-meta-history3.xml-p4040466p4166564.tsv","61-meta-history3.xml-p4166565p4310744.tsv","62-meta-history3.xml-p4310745p4419858.tsv","63-meta-history4.xml-p4419861p4565226.tsv","64-meta-history4.xml-p4565227p4699279.tsv","65-meta-history4.xml-p4699280p4830738.tsv","66-meta-history4.xml-p4830739p4970702.tsv","69-meta-history4.xml-p5314076p5483062.tsv",
"70-meta-history4.xml-p5483063p5659533.tsv","71-meta-history4.xml-p5659534p5847437.tsv","72-meta-history4.xml-p5847438p6039053.tsv"]

len(full_tsv_list)

#'64-meta-history4.xml-p4565227p4699279.tsv' is not processable

71

In [15]:
# define input files set (eg: the 5 first elements of full_tsv_list)
# one can also iterate over groups of 5 files

list_files = full_tsv_list[0:5]

# number of sample edits to be drawn from each tsv file
n_user = 20

# seed for random sampling, set None for undefined seed
random_nbr = 73

# if list_files containes 5 elements and n_user is 20,
# df_sample will contain 5*20 = 100 samples

In [29]:
t0 = time()


path = '/media/hdd/salaun/wikiedit/tsv_output/' # path of tsv files

try:
    del df_sample
    print('df_sample deleted')
except:
    print('No df_sample')
try:
    del df_full
    print('df_full deleted')
except:
    print('No df_full')

print("Total number of files to be processed:", len(list_files))

init = True # initialize

for file in list_files:
    
    t1 = time()
    
    print('\n')
    print('Processing input file:', file)
    print('Running...')
    df_full = pd.read_csv(path+file, sep ='\t')
    
    ##### START FILTERS
    print('Shape before filtering: {0:,} rows | {1:,} columns'.format(df_full.shape[0], df_full.shape[1]))
    
    # remove version pairs with empty strings
    df_full = df_full.fillna('')
    df_full = df_full[(df_full.modif_remove != "") & (df_full.modif_add != "")]
    for element in [' ', '\n']:
        df_full = df_full[~df_full.modif_remove.str.startswith(element) & ~df_full.modif_add.str.startswith(element)]
    
    # remove samples whose title shows the page is editor-oriented and not reader-oriented
    for element in ['Discussion', 'Projet', 'Utilisateur', 'Wikipédia', 'Modèle', 'Catégorie']:
        df_full = df_full[~df_full.title.str.startswith(element)]

    # remove samples whose comments mention spelling or typing mistakes
    for element in ['ortho', 'gramm', 'typo', 'frappe']:
        df_full = df_full[~df_full.comment.str.lower().str.contains(element)]
    
    # remove samples whose comments mention reverting
    for element in ['Annulation des modifications', 'Révocation des modifications']:
        df_full = df_full[~df_full.comment.str.contains(element)]
    
    # remove sample with minor binary set as true
    df_full = df_full[~df_full.minor]
    
    # remove samples whose username suggests the editor is a bot
    df_full = df_full[~df_full.username.str.lower().str.contains("bot") & ~df_full.comment.str.lower().str.contains("bot ")]
    
    # remove samples where modifications concern formatting (hyperlinks, functions, section title, 
    # bullet list, box, numbered list, references, math formulas, figures, tables, files)
    for s in ['[', ']', '{', '}', '==', '===', '====', '=====', '*', '|', ';', '#', '!', ':', '<']: #,'•'
        df_full = df_full[~df_full.modif_remove.str.startswith(s) & ~df_full.modif_add.str.startswith(s)]
   
    for s in ['<ref', 'ref>', '<math>', '</math>', 'Fichier:', 'Image:', '\n\\|', '<br', 'br>', 'refnec']: #,'<sup', 'sup>', '<sub', 'sub>']:  ,
        df_full = df_full[~df_full.modif_remove.str.contains(s) & ~df_full.modif_add.str.contains(s)]
    
    print('Shape after filtering: {0:,} rows | {1:,} columns'.format(df_full.shape[0], df_full.shape[1]))
    ##### END FILTERS
    
    if init:
        df_sample = df_full.sample(n=n_user, random_state=random_nbr)
        df_sample = df_sample.fillna('')
        
    else:
        df_sample_part = df_full.sample(n=n_user, random_state=random_nbr)
        df_sample_part = df_sample_part.fillna('')
        df_sample = pd.concat([df_sample, df_sample_part], ignore_index=True)
        
    init = False
    del df_full
    
    t2 = time()-t1
    m, s = divmod(t2, 60)
    print('Round runtime: %d:%02d' % (m, s))

df_sample.shape

i=0

# create sample_id
for row in range(0, len(df_sample)):
    df_sample.loc[row,'sample_id'] = df_sample.loc[row,'id_file'] + '_' + str(df_sample.loc[row,'id_modif'])
    i+=1
print(i)

df_sample = df_sample[['sample_id', 'id_file', 'id_modif', 'label_incoherence', 'username', 'user_id',
       'registered', 'format', 'model', 'id', 'parentid', 'title', 'minor',
       'comment', 'modif', 'modif_remove', 'modif_add', 'timestamp',
       'filt_bot', 'filt_coher', 'filt_confli', 'filt_contradic',
       'filt_erreur', 'filt_erron', 'filt_faux', 'filt_frappe', 'filt_gramma',
       'filt_ortho', 'filt_revert', 'filt_sens', 'filt_tromp', 'filt_typo',
       'filt_vandalisme', 'filt_vraise']]

duration = time() - t0
m, s = divmod(duration, 60)
h, m = divmod(m, 60)
print("Running time: %d:%02d:%02d" % (h, m, s), '\n')

df_sample deleted
No df_full
Total number of files: 5


Processing input file: 35-meta-history2.xml-p1113861p1186656.tsv
Running...
Shape before filtering: 1,499,689 | 34
Shape after filtering: 112,736 | 34
Round runtime: 1:12


Processing input file: 36-meta-history2.xml-p1186657p1252585.tsv
Running...
Shape before filtering: 1,498,372 | 34
Shape after filtering: 116,564 | 34
Round runtime: 1:05


Processing input file: 37-meta-history2.xml-p1252586p1312869.tsv
Running...
Shape before filtering: 1,499,250 | 34
Shape after filtering: 101,767 | 34
Round runtime: 1:05


Processing input file: 38-meta-history2.xml-p1312870p1382438.tsv
Running...
Shape before filtering: 1,497,167 | 34
Shape after filtering: 112,565 | 34
Round runtime: 1:10


Processing input file: 39-meta-history2.xml-p1382439p1454699.tsv
Running...
Shape before filtering: 1,499,709 | 34
Shape after filtering: 94,256 | 34
Round runtime: 1:15


(100, 34)

100
Running time: 0:05:50 



In [ ]:
df_sample.head()

In [ ]:
# save samples in a special format for human-made annotation tasks
pre_annot_file = '35_39_73_pre_annot.csv'
df_sample.to_csv('/media/hdd/salaun/wikiedit/annotations/data_v1/'+ pre_annot_file, sep=';')

# save samples as df for classification experiments
sample_tsv_file = '35_39_73_sample.tsv'
df_sample.to_csv('/media/hdd/salaun/wikiedit/annotations/data_v1/'+ sample_tsv_file, sep='\t')